In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import geemap

import geeft

In [3]:
ee.Initialize()

In [4]:
from helpers import LandSAT8SR
from helpers import AOI
from helpers import DATES

In [5]:
ee.Initialize()

In [6]:
HARMONICS = 3

frequencies = list(range(1, HARMONICS + 1))

In [7]:
cos_names = geeft._get_names('cos', frequencies)
sin_names = geeft._get_names('sin', frequencies)

In [8]:
independents = ['constant', 't'] + cos_names + sin_names
dependent = 'NDVI'

In [9]:
# Computes Harmonics
dataset = (
    LandSAT8SR()
    .filterBounds(AOI)
    .filterDate(*DATES)
    .applyCloudMask()
    .addNDVI()
    .map(geeft._add_constant)
    .map(geeft._add_time)
    .map(geeft._add_harmonics(cos_names=cos_names, sin_names=sin_names))
)

In [10]:
trend = geeft._compute_trend(dataset, dependent, independents)

In [11]:
trend

In [12]:
# compute coeff
coef = geeft._compute_coefficients(trend, independents)

In [13]:
coef

In [14]:
# add coefficients to each image
dataset = dataset.map(geeft._add_coef(coef))


In [15]:
dataset.first().bandNames()

In [16]:
for mode in frequencies:
    dataset = dataset.map(geeft._add_phase(mode)).map(geeft._add_amplitude(mode))

In [17]:
dataset.first().bandNames()

In [18]:
dataset.first().bandNames()

In [19]:
dataset = dataset.map(geeft._add_phase(3))

In [20]:
dataset.first().bandNames()